### 1. Importing Libraries

In [3]:
import os
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split 
import warnings

warnings.filterwarnings("ignore")

### 2. Reading Data

In [3]:
PROJECT_DIR = r'/Users/sukhsodhi/Desktop/DS_DA/Projects/flight_prediction_sagemaker'
DATA_DIR = 'Data'

In [4]:
def get_data(name):
    
    file_name = f'{name}.csv'
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    return pd.read_csv(file_path)

In [10]:
data = get_data('flight_price')

In [11]:
data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


## 3. Preliminary Analysis 

### 3.1 Data Types

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- Dataset has `10683` observations and `11` features.

### 3.2 Check Duplicates

In [15]:
data.duplicated().sum()

220

In [23]:
(
    data[data.duplicated(keep=False)]
    .sort_values(['Airline', 'Date_of_Journey', 'Source', 'Destination'])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### 3.3 Observations



**Column Transformation**.     
- Date_of_Journey, Arrival_Time, Dep_Time --> `Datetime`   
- Transform information in `Duration` column. Make numeric data type out of it.
- Date type of `Total_Stops` should be numeric.
- Dataset has `220` duplicate values.

## 4. Column wise Data Checking

**Airlines column**

In [60]:
data['Airline'].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

**Convert**
- Jet Airways Business to Jet Airways.
- Vistara Premium economy to Vistara.
- Multiple carriers Premium economy to Multiple carriers.
- Jet Airways Business to Jet Airways.

In [69]:
(
    data['Airline']
    .str.replace(' Premium economy', '')
    .str.replace(' Business', '')
    .str.title()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

In [132]:
data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [133]:
data['Source'].unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [134]:
data['Destination'].unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

In [151]:
(
    data['Arrival_Time']
    .loc[lambda x: x.str.contains('[^0-9:]')]
    .str.split(" ", n=1)
    .str.get(-1)
    .unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

In [173]:
(
    data['Duration']
    .loc[lambda x: ~x.str.contains('m')]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [175]:
(
    data['Duration']
    .loc[lambda x: ~x.str.contains('h')]
#     .unique()
)

6474    5m
Name: Duration, dtype: object

In [178]:
data.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


In [ ]:
# No flight can be of duration 5 minutes 

In [206]:
(
    data['Duration']
    .drop(index = 6474)
    .str.split(' ', expand = True)
    .set_axis(['Hour', 'Minute'], axis = 1)
    .assign
    (
        Hour = lambda df_: df_['Hour'].str.replace('h', '')
        .astype(int)
        .mul(60),
        
        Minute = lambda df_ : df_['Minute'].str.replace('m', '')
        .fillna('0')
        .astype(int)
    )
    .sum(axis=1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [221]:
data['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

## 5. Cleaning Operations

In [218]:
def convert_to_minutes(series):
    return 
    (
        series
            .str.split(' ', expand = True)
            .set_axis(['Hour', 'Minute'], axis = 1)
            .assign
            (
                Hour = lambda df_: 
                (
                    df_['Hour']
                    .str.replace('h', '')
                    .astype(int)
                    .mul(60)
                ),

                Minute = lambda df_ : 
                (
                    df_['Minute']
                    .str.replace('m', '')
                    .fillna('0')
                    .astype(int)
                )
            )
            .sum(axis=1)
    )

In [231]:
def clean_data(df):
    return (
        df
        .drop(index=[6474])
        .assign(**{
            col: df[col].str.strip() 
            for col in df.select_dtypes(include='O').columns
        }) # Removing white spaces 
        
        .rename(columns=str.lower) # Lower casing the columns
        
        .assign # Transforming Airline Column
        (
            airline = 
            (
                df['Airline']
                .str.replace(' Premium economy', '', case=False)
                .str.replace(' Business', '', case=False)
                .str.replace('Goair', 'Go Air', case=False)
                .str.title()
            ),
            
            date_of_journey = pd.to_datetime(df['Date_of_Journey'], dayfirst=True),
            dep_time = lambda df_ : pd.to_datetime(df_['dep_time'], format='%H:%M').dt.time,
            arrival_time = lambda df_ : pd.to_datetime(df_['arrival_time']).dt.time,
            
            duration = lambda df_ : 
            (
                df_['duration'].pipe(convert_to_minutes)
            ),
            
            total_stops = lambda df_ :
            (
                df_['total_stops']
                .str.replace('non-stop', '0')
                .str.replace(' stops?', '', regex = True)
                .pipe(lambda ser : pd.to_numeric(ser))
            )
            
            
        )
        .drop(columns = 'route')

    )


In [235]:
cleaned_data = clean_data(data)
cleaned_data.head()

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No info,13302


## 6. Splitting Data

In [240]:
final_data = cleaned_data

X = final_data.drop(columns="price")
y = final_data.price.copy()


X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)


(6836, 9) (6836,)
(1709, 9) (1709,)
(2137, 9) (2137,)


## 7. Exporting Datasets

In [241]:
def export_data(X, y, name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)

    X.join(y).to_csv(file_path, index=False)

    return pd.read_csv(file_path).head()


export_data(X_train, y_train, "train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-03,Delhi,Cochin,02:15:00,04:25:00,1570,1.0,No info,17024
1,Vistara,2019-03-24,Kolkata,Banglore,07:10:00,18:45:00,695,1.0,No info,16932
2,Spicejet,2019-04-09,Banglore,Delhi,09:30:00,12:20:00,170,0.0,No info,4423
3,Indigo,2019-04-27,Banglore,Delhi,21:15:00,00:15:00,180,0.0,No info,3943
4,Air India,2019-06-12,Delhi,Cochin,09:45:00,09:25:00,1420,1.0,No info,7480


In [242]:
export_data(X_val, y_val, "val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-12,Kolkata,Banglore,16:30:00,12:00:00,1170,1.0,In-flight meal not included,8529
1,Jet Airways,2019-05-18,Banglore,Delhi,11:10:00,14:05:00,175,0.0,In-flight meal not included,5198
2,Multiple Carriers,2019-06-12,Delhi,Cochin,08:45:00,19:00:00,615,1.0,No info,11789
3,Multiple Carriers,2019-04-24,Delhi,Cochin,07:10:00,16:10:00,540,1.0,In-flight meal not included,6093
4,Jet Airways,2019-03-27,Delhi,Cochin,15:05:00,04:25:00,800,1.0,No info,12242


In [243]:
export_data(X_test, y_test, "test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Multiple Carriers,2019-05-21,Delhi,Cochin,15:05:00,01:30:00,625,2.0,No info,16655
1,Go Air,2019-06-03,Delhi,Cochin,10:35:00,19:35:00,540,1.0,No info,4959
2,Vistara,2019-05-09,Kolkata,Banglore,20:20:00,09:05:00,765,1.0,No info,9187
3,Indigo,2019-05-24,Chennai,Kolkata,14:45:00,17:05:00,140,0.0,No info,3858
4,Jet Airways,2019-05-21,Delhi,Cochin,22:50:00,04:25:00,335,1.0,In-flight meal not included,12898
